<a href="https://colab.research.google.com/github/noctics123/dm_pi_ruben_adv_de/blob/main/Scrapping_Macroeconomics_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests pandas


In [2]:
!pip install requests

## **INDICADOR DESEMPLEO**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import when, col, create_map, lit
import requests
import json
from pyspark.sql.functions import broadcast

# Initialize Spark session
builder = SparkSession.builder \
    .appName("WorldBankData360Monthly") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()
spark = builder

# Define the schema with only the required fields
schema = StructType([
    StructField("OBS_VALUE", DoubleType(), True),  # Tasa
    StructField("TIME_PERIOD", StringType(), True),  # Año
    StructField("FREQ", StringType(), True),  # Periodicidad code
    StructField("REF_AREA", StringType(), True)  # País ID
])

# Country code to country name mapping
country_mapping = {
    "DOM": "Dominican Republic",
    "MAR": "Morocco",
    "AGO": "Angola",
    "ABW": "Aruba",
    "AFG": "Afghanistan",
    "ALB": "Albania",
    "ARE": "United Arab Emirates",
    "ARG": "Argentina",
    "ARM": "Armenia",
    "ATG": "Antigua and Barbuda",
    "AUS": "Australia",
    "AUT": "Austria",
    "AZE": "Azerbaijan",
    "BDI": "Burundi",
    "BEL": "Belgium",
    "BEN": "Benin",
    "BFA": "Burkina Faso",
    "BGD": "Bangladesh",
    "BGR": "Bulgaria",
    "BHR": "Bahrain",
    "BHS": "Bahamas, The",
    "BIH": "Bosnia and Herzegovina",
    "BLR": "Belarus",
    "BLZ": "Belize",
    "BOL": "Bolivia",
    "BRA": "Brazil",
    "BRB": "Barbados",
    "BRN": "Brunei Darussalam",
    "BTN": "Bhutan",
    "BWA": "Botswana",
    "CAN": "Canada",
    "CHE": "Switzerland",
    "CHL": "Chile",
    "CHN": "China",
    "CIV": "Cote d'Ivoire",
    "CMR": "Cameroon",
    "COD": "Congo, Dem. Rep.",
    "COG": "Congo, Rep.",
    "COL": "Colombia",
    "COM": "Comoros",
    "CPV": "Cabo Verde",
    "CRI": "Costa Rica",
    "CUW": "Curacao",
    "CYP": "Cyprus",
    "CZE": "Czechia",
    "DEU": "Germany",
    "DJI": "Djibouti",
    "DMA": "Dominica",
    "DNK": "Denmark",
    "DZA": "Algeria",
    "ECU": "Ecuador",
    "EGY": "Egypt, Arab Rep.",
    "ESP": "Spain",
    "EST": "Estonia",
    "ETH": "Ethiopia",
    "FIN": "Finland",
    "FJI": "Fiji",
    "FRA": "France",
    "FSM": "Micronesia, Fed. Sts.",
    "GAB": "Gabon",
    "GBR": "United Kingdom",
    "GEO": "Georgia",
    "GHA": "Ghana",
    "GIN": "Guinea",
    "GMB": "Gambia, The",
    "GNB": "Guinea-Bissau",
    "GNQ": "Equatorial Guinea",
    "GRC": "Greece",
    "GRD": "Grenada",
    "GTM": "Guatemala",
    "GUY": "Guyana",
    "HKG": "Hong Kong SAR, China",
    "HND": "Honduras",
    "HRV": "Croatia",
    "HTI": "Haiti",
    "HUN": "Hungary",
    "IDN": "Indonesia",
    "IND": "India",
    "IRL": "Ireland",
    "IRN": "Iran, Islamic Rep.",
    "IRQ": "Iraq",
    "ISL": "Iceland",
    "ISR": "Israel",
    "ITA": "Italy",
    "JAM": "Jamaica",
    "JOR": "Jordan",
    "JPN": "Japan",
    "KAZ": "Kazakhstan",
    "KEN": "Kenya",
    "KGZ": "Kyrgyz Republic",
    "KHM": "Cambodia",
    "KIR": "Kiribati",
    "KNA": "St. Kitts and Nevis",
    "KOR": "Korea, Rep.",
    "KWT": "Kuwait",
    "LAO": "Lao PDR",
    "LBN": "Lebanon",
    "LBR": "Liberia",
    "LBY": "Libya",
    "LCA": "St. Lucia",
    "LKA": "Sri Lanka",
    "LSO": "Lesotho",
    "LTU": "Lithuania",
    "LUX": "Luxembourg",
    "LVA": "Latvia",
    "MAC": "Macao SAR, China",
    "MDA": "Moldova",
    "MDG": "Madagascar",
    "MDV": "Maldives",
    "MEX": "Mexico",
    "MHL": "Marshall Islands",
    "MKD": "North Macedonia",
    "MLI": "Mali",
    "MLT": "Malta",
    "MMR": "Myanmar",
    "MNE": "Montenegro",
    "MNG": "Mongolia",
    "MOZ": "Mozambique",
    "MRT": "Mauritania",
    "MUS": "Mauritius",
    "MWI": "Malawi",
    "MYS": "Malaysia",
    "NAM": "Namibia",
    "NER": "Niger",
    "NGA": "Nigeria",
    "NIC": "Nicaragua",
    "NLD": "Netherlands",
    "NOR": "Norway",
    "NPL": "Nepal",
    "NZL": "New Zealand",
    "OMN": "Oman",
    "PAK": "Pakistan",
    "PAN": "Panama",
    "PER": "Peru",
    "PHL": "Philippines",
    "PLW": "Palau",
    "PNG": "Papua New Guinea",
    "POL": "Poland",
    "PRT": "Portugal",
    "PRY": "Paraguay",
    "PSE": "West Bank and Gaza",
    "QAT": "Qatar",
    "ROU": "Romania",
    "RUS": "Russian Federation",
    "RWA": "Rwanda",
    "SAU": "Saudi Arabia",
    "SDN": "Sudan",
    "SEN": "Senegal",
    "SGP": "Singapore",
    "SLB": "Solomon Islands",
    "SLE": "Sierra Leone",
    "SLV": "El Salvador",
    "SMR": "San Marino",
    "SOM": "Somalia",
    "SRB": "Serbia",
    "STP": "Sao Tome and Principe",
    "SUR": "Suriname",
    "SVK": "Slovak Republic",
    "SVN": "Slovenia",
    "SWE": "Sweden",
    "SWZ": "Eswatini",
    "SYC": "Seychelles",
    "SYR": "Syrian Arab Republic",
    "TCD": "Chad",
    "TGO": "Togo",
    "THA": "Thailand",
    "TJK": "Tajikistan",
    "TLS": "Timor-Leste",
    "TON": "Tonga",
    "TTO": "Trinidad and Tobago",
    "TUN": "Tunisia",
    "TUR": "Turkiye",
    "TUV": "Tuvalu",
    "TZA": "Tanzania",
    "UGA": "Uganda",
    "UKR": "Ukraine",
    "URY": "Uruguay",
    "USA": "United States",
    "UZB": "Uzbekistan",
    "VCT": "St. Vincent and the Grenadines",
    "VEN": "Venezuela, RB",
    "VNM": "Viet Nam",
    "VUT": "Vanuatu",
    "YEM": "Yemen, Rep.",
    "ZAF": "South Africa",
    "ZMB": "Zambia",
    "ZWE": "Zimbabwe"
}

# API endpoint and base parameters (filtered for monthly data)
url = "https://data360api.worldbank.org/data360/data"
params_base = {
    "DATABASE_ID": "IMF_IFS",
    "INDICATOR": "IMF_IFS_LUR",
    "timePeriodFrom": "1949-01",
    "timePeriodTo": "2024-12",
    "FREQ": "M"  # Filter for monthly data
}

# Initialize an empty list to store all data
all_data = []

# Pagination loop
skip = 0
while True:
    params = params_base.copy()
    params["skip"] = skip

    # Make the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        values = data.get("value", [])
        if not values:  # Break if no more data is returned
            break
        all_data.extend(values)
        skip += 1000  # Increment skip for the next batch
        print(f"Fetched batch with {len(values)} records, total so far: {len(all_data)}")
        if skip == 0:  # Print raw data from the first batch for debugging
            print("Sample raw data from first batch:", json.dumps(values[0], indent=2))
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

# Pre-process data to convert OBS_VALUE to float, handling potential errors
print(f"Total records before processing: {len(all_data)}")
for i, item in enumerate(all_data):
    try:
        if item.get("OBS_VALUE") is not None:
            item["OBS_VALUE"] = float(item["OBS_VALUE"])
    except (ValueError, TypeError) as e:
        print(f"Error converting OBS_VALUE at index {i}: {e}, setting to None")
        item["OBS_VALUE"] = None  # Set to None if conversion fails
print(f"Total records after processing: {len(all_data)}")

# Convert the data to a Spark DataFrame with the defined schema
try:
    df_spark = spark.createDataFrame(all_data, schema=schema)
    print("DataFrame created successfully.")
except Exception as e:
    print(f"Error creating DataFrame: {e}")
    raise

# Add Freq Name column by mapping FREQ codes to full names
df_spark = df_spark.withColumn("Freq Name",
                              when(df_spark.FREQ == "A", "Anual")
                              .when(df_spark.FREQ == "M", "Mensual")
                              .when(df_spark.FREQ == "Q", "Trimestral")
                              .otherwise("Desconocido"))

# Create a mapping DataFrame for country codes to names
country_map_df = spark.createDataFrame(list(country_mapping.items()), ["REF_AREA", "País"])

# Join the main DataFrame with the country mapping DataFrame
df_spark = df_spark.join(broadcast(country_map_df), "REF_AREA", "left_outer")

# Select and rename columns to match your requirements
df_spark = df_spark.select(
    df_spark.TIME_PERIOD.alias("Año"),
    df_spark.FREQ.alias("Periodicidad"),
    df_spark["Freq Name"].alias("Freq Name"),
    df_spark.OBS_VALUE.alias("Tasa"),
    df_spark.REF_AREA.alias("País_ID"),
    df_spark.País.alias("País")
)

# Count the total number of rows
try:
    row_count = df_spark.count()
    print(f"Total number of rows in the DataFrame: {row_count}")
except Exception as e:
    print(f"Error counting rows: {e}")
    raise

# Write the DataFrame to a Parquet file
parquet_path = "parquet/unemployment_data_monthly"
df_spark.write \
    .mode("overwrite") \
    .partitionBy("Año") \
    .parquet(parquet_path)
print(f"Parquet file saved to {parquet_path}")

# Read the Parquet file back
try:
    df_read = spark.read.parquet(parquet_path)
    print("Parquet file read successfully.")
    df_read.printSchema()
    df_read.show()
except Exception as e:
    print(f"Error reading Parquet file: {e}")
    raise

# Stop the Spark session
spark.stop()

Fetched batch with 1000 records, total so far: 1000
Fetched batch with 1000 records, total so far: 2000
Fetched batch with 1000 records, total so far: 3000
Fetched batch with 1000 records, total so far: 4000
Fetched batch with 1000 records, total so far: 5000
Fetched batch with 1000 records, total so far: 6000
Fetched batch with 1000 records, total so far: 7000
Fetched batch with 1000 records, total so far: 8000
Fetched batch with 1000 records, total so far: 9000
Fetched batch with 1000 records, total so far: 10000
Fetched batch with 1000 records, total so far: 11000
Fetched batch with 1000 records, total so far: 12000
Fetched batch with 1000 records, total so far: 13000
Fetched batch with 1000 records, total so far: 14000
Fetched batch with 1000 records, total so far: 15000
Fetched batch with 1000 records, total so far: 16000
Fetched batch with 1000 records, total so far: 17000
Fetched batch with 1000 records, total so far: 18000
Fetched batch with 1000 records, total so far: 19000
Fe

## **INDICADOR TIPO DE CAMBIO**

In [4]:
pip install pyspark


In [5]:
pip install requests pandas python-dateutil


In [6]:
import requests

# API endpoint (corrected URL)
base_url = "https://desarrollador.sb.gob.do/api/estadísticas-del-sistema-financiero-con-paginacion/captacionesLocalidad"

# API keys
primary_key = "dff2111ccbcd44c2ac47265b658949f3"
secondary_key = "f259f9637e9343538cf1b06770aaae25"

# Headers for authentication
headers = {
    "Ocp-Apim-Subscription-Key": primary_key,
    "Content-Type": "application/json"
}

# Pagination parameters
params = {
    "page": 1,
    "pageSize": 100
}

# Initialize data storage
all_data = []

# Try with primary key
try:
    print("Trying with primary key...")
    response = requests.get(base_url, headers=headers, params=params)
    response.raise_for_status()  # Raise error for non-200 status
    data = response.json()
    all_data.extend(data.get("results", []))
    print(f"Retrieved {len(data.get('results', []))} records with primary key")
except requests.exceptions.HTTPError as e:
    print(f"Primary key failed: {e}")

    # Try with secondary key
    headers["Ocp-Apim-Subscription-Key"] = secondary_key
    try:
        print("Trying with secondary key...")
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        all_data.extend(data.get("results", []))
        print(f"Retrieved {len(data.get('results', []))} records with secondary key")
    except requests.exceptions.HTTPError as e:
        print(f"Secondary key failed: {e}")
        print("Both keys failed. Check API documentation or contact support: soporteapis@sb.gob.do")
        exit()

# Handle pagination
while "nextPage" in data and data["nextPage"]:
    params["page"] += 1
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        all_data.extend(data.get("results", []))
        print(f"Retrieved page {params['page']} with {len(data.get('results', []))} records")
    except requests.exceptions.HTTPError as e:
        print(f"Error fetching page {params['page']}: {e}")
        break

# Save data to CSV (optional)
if all_data:
    import pandas as pd
    df = pd.DataFrame(all_data)
    df.to_csv("captaciones_localidad.csv", index=False)
    print(f"Total records retrieved: {len(all_data)}. Saved to captaciones_localidad.csv")
else:
    print("No data retrieved. Verify endpoint, keys, or permissions.")

# Print sample data (first few records)
for record in all_data[:5]:  # Show up to 5 records
    print(record)

Trying with primary key...
Primary key failed: 403 Client Error: Forbidden for url: https://desarrollador.sb.gob.do/api/estad%C3%ADsticas-del-sistema-financiero-con-paginacion/captacionesLocalidad?page=1&pageSize=100
Trying with secondary key...
Secondary key failed: 403 Client Error: Forbidden for url: https://desarrollador.sb.gob.do/api/estad%C3%ADsticas-del-sistema-financiero-con-paginacion/captacionesLocalidad?page=1&pageSize=100
Both keys failed. Check API documentation or contact support: soporteapis@sb.gob.do
No data retrieved. Verify endpoint, keys, or permissions.


In [1]:
pip install selenium webdriver-manager pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.1 MB/s eta 0:00:00
